In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# Define dataset path (Kaggle specific)
data_dir = "/kaggle/input/dataset/dataset/dataset"  # Update with actual Kaggle dataset path
labels_file = "/kaggle/input/dataset/metaData.xlsx"

In [5]:
# Load labels
labels_df = pd.read_excel(labels_file)
label_dict = {row["Folder Name"]: row["Char Name"] for _, row in labels_df.iterrows()}

In [6]:
# Parameters
img_size = (32, 32)
batch_size = 32
epochs = 15  # Optimized for Kaggle free tier
num_classes = 122  # Total classes from dataset

In [7]:
# Image Data Generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [8]:
# Load dataset
def create_generators():
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, val_generator

train_generator, val_generator = create_generators()

Found 293664 images belonging to 122 classes.
Found 73354 images belonging to 122 classes.


In [9]:
# Model using MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights=None)
base_model.trainable = True

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3082/9177 ━━━━━━━━━━━━━━━━━━━━ 12:02 119ms/step - accuracy: 0.0279 - loss: 4.5752

In [ ]:
# Save the model to Kaggle working directory
model.save("/kaggle/working/ekush_character_recognition.h5")